# Model Management

This notebook will demonstrate the model management step by using BluePyMM.

Model management consists of three phases:
1. [**prepare**](#prepare): processing of input data, finding all possible morphology/electrical model combinations (*me-combinations*), and preparation of a database
2. [**run**](#run): run all me-combinations
3. [**select**](#select): compare all me-combinations against input thresholds, select successful combinations and write them out to file; generate report

## <a id='prepare'>Prepare me-combinations</a>

In the preparation phase, the input data is processed, which consists of morphology releases, a set of released e-models and a recipe that describes which layers, morphology types (*m-types*), and electrical types (*e-types*) should be tested.

In [1]:
import pandas as pd
from pathlib import Path
import sqlite3

from bluepymm import prepare_combos, run_combos, select_combos

In [2]:
prepare_config = 'prepare.json'
prepare_combos.prepare_combos(conf_filename=prepare_config, continu=False)


Reading configuration at prepare.json
Getting final emodels dict
Preparing emodels in /home/anil/git-repos/SSCxEModelExamples/model_management/mm_run_minimal/tmp/emodels
Parallelising preparation of e-model directories
Preparing: cADpyr_L5TPC
Compiling mechanisms ...
Creating sqlite db at /home/anil/git-repos/SSCxEModelExamples/model_management/mm_run_minimal/output/scores.sqlite
Using repaired exemplar morph path: ./morphologies
Using unrepaired exemplar morph path: ./morphologies
Reading recipe at ./cell_composition.yaml
Reading neuronDB at ./morphologies/neuronDB.xml
Reading repaired-morphologies neuronDB at ./morphologies/neuronDB.xml
Merging recipe and neuronDB tables
Creating emodel etype table
Creating full table by merging subtables
Filtering out morp_names that dont match regex
Adding exemplar rows
Adding exemplar row for e-model cADpyr_L5TPC
Created sqlite db at /home/anil/git-repos/SSCxEModelExamples/model_management/mm_run_minimal/output/scores.sqlite


The sqlite database in the output folder contains the tabular information on the etype and morphology required for the run phase of BluePyMM.

In [3]:
db_path = Path("output") / "scores.sqlite"
db_connection = sqlite3.connect(db_path)

In [4]:
pd.read_sql_query("SELECT * from scores", db_connection)

,index,layer,fullmtype,mtype,msubtype,etype,morph_name,morph_ext,emodel,original_emodel,morph_dir,scores,opt_scores,exception,to_run,is_exemplar,is_repaired,is_original,submtype,extra_values
0,0,None,L5_TPC:A,L5_TPC,A,cADpyr,C060114A5,.asc,cADpyr_L5TPC,cADpyr_L5TPC,./morphologies,None,None,None,1,1,1,0,None,None
1,1,None,L5_TPC:A,L5_TPC,A,cADpyr,C060114A5,.asc,cADpyr_L5TPC,cADpyr_L5TPC,./morphologies,None,None,None,1,1,1,1,None,None
2,2,None,L5_TPC:A,L5_TPC,A,cADpyr,C060114A5,.asc,cADpyr_L5TPC,cADpyr_L5TPC,./morphologies,None,"{""L5TPCa.Step_280.soma.v.inv_fourth_ISI"": 2.26...",None,1,1,0,0,None,None
3,3,None,L5_TPC:A,L5_TPC,A,cADpyr,C060114A5,.asc,cADpyr_L5TPC,cADpyr_L5TPC,./morphologies,None,"{""L5TPCa.Step_280.soma.v.inv_fourth_ISI"": 2.26...",None,1,1,0,1,None,None
4,4,5,L5_TPC:A,L5_TPC,None,cADpyr,C060110A2,None,cADpyr_L5TPC,cADpyr_L5TPC,./morphologies,None,None,None,1,0,1,0,A,None
5,5,5,L5_TPC:A,L5_TPC,None,cADpyr,rat_20140925_RH1_Exp1_cell2,None,cADpyr_L5TPC,cADpyr_L5TPC,./morphologies,None,None,None,1,0,1,0,A,None
6,6,5,L5_TPC:A,L5_TPC,None,cADpyr,rat_20160316_E2_LH3_cell1,None,cADpyr_L5TPC,cADpyr_L5TPC,./morphologies,None,None,None,1,0,1,0,A,None
7,7,5,L5_TPC:A,L5_TPC,None,cADpyr,rp110616_C1_idB,None,cADpyr_L5TPC,cADpyr_L5TPC,./morphologies,None,None,None,1,0,1,0,A,None
8,8,5,L5_TPC:A,L5_TPC,None,cADpyr,rp110630_P1_idC,None,cADpyr_L5TPC,cADpyr_L5TPC,./morphologies,None,None,None,1,0,1,0,A,None
9,9,5,L5_TPC:A,L5_TPC,None,cADpyr,C060114A5,None,cADpyr_L5TPC,cADpyr_L5TPC,./morphologies,None,None,None,1,0,1,0,A,None


## <a id='run'>Run me-combinations</a>

In this phase, all the me-combinations as found in the SQLite database are run. The resulting scores are saved for later processing.

In [5]:
run_config = 'run.json'
run_combos.run_combos(conf_filename=run_config)

Reading configuration at run.json
Calculating scores
Creating argument list for parallelisation
Found 10 rows in score database to run
Parallelising score evaluation of 10 me-combos
Saved scores for uid 5 (1 out of 10) 
Saved scores for uid 7 (2 out of 10) 
Saved scores for uid 4 (3 out of 10) 
Saved scores for uid 6 (4 out of 10) 
Saved scores for uid 8 (5 out of 10) 
Saved scores for uid 0 (6 out of 10) 
Saved scores for uid 3 (7 out of 10) 
Saved scores for uid 2 (8 out of 10) 
Saved scores for uid 1 (9 out of 10) 
Saved scores for uid 9 (10 out of 10) 
Converting score json strings to scores values ...


Now if we look at the previous scores.sqlite database, we will notice the scores and extra_values columns are filled.

In [6]:
pd.read_sql_query("SELECT morph_name, emodel, scores, extra_values from scores", db_connection)

,morph_name,emodel,scores,extra_values
0,C060114A5,cADpyr_L5TPC,"{""mm.bAP.soma.v.Spikecount"": 0.0, ""mm.bAP.dend...","{""holding_current"": -0.17086359880323404, ""thr..."
1,C060114A5,cADpyr_L5TPC,"{""mm.bAP.soma.v.Spikecount"": 0.0, ""mm.bAP.dend...","{""holding_current"": -0.17086359880323404, ""thr..."
2,C060114A5,cADpyr_L5TPC,"{""mm.bAP.soma.v.Spikecount"": 0.0, ""mm.bAP.dend...","{""holding_current"": -0.17086359880323404, ""thr..."
3,C060114A5,cADpyr_L5TPC,"{""mm.bAP.soma.v.Spikecount"": 0.0, ""mm.bAP.dend...","{""holding_current"": -0.17086359880323404, ""thr..."
4,C060110A2,cADpyr_L5TPC,"{""mm.bAP.soma.v.Spikecount"": 0.0, ""mm.bAP.dend...","{""holding_current"": -0.13266207293887194, ""thr..."
5,rat_20140925_RH1_Exp1_cell2,cADpyr_L5TPC,"{""mm.bAP.soma.v.Spikecount"": 0.0, ""mm.bAP.dend...","{""holding_current"": -0.04746268686018198, ""thr..."
6,rat_20160316_E2_LH3_cell1,cADpyr_L5TPC,"{""mm.bAP.soma.v.Spikecount"": 0.0, ""mm.bAP.dend...","{""holding_current"": -0.03704371428680028, ""thr..."
7,rp110616_C1_idB,cADpyr_L5TPC,"{""mm.bAP.soma.v.Spikecount"": 0.0, ""mm.bAP.dend...","{""holding_current"": -0.11598425440488296, ""thr..."
8,rp110630_P1_idC,cADpyr_L5TPC,"{""mm.bAP.soma.v.Spikecount"": 0.0, ""mm.bAP.dend...","{""holding_current"": -0.16892891194503407, ""thr..."
9,C060114A5,cADpyr_L5TPC,"{""mm.bAP.soma.v.Spikecount"": 0.0, ""mm.bAP.dend...","{""holding_current"": -0.17086359880323404, ""thr..."


The scores are also stored in the score_values table with each score having a different column for convenience.

In [7]:
pd.read_sql_query("SELECT * from score_values", db_connection)

,mm.bAP.soma.v.Spikecount,mm.bAP.dend1.v.maximum_voltage_from_voltagebase,mm.bAP.dend2.v.maximum_voltage_from_voltagebase,mm.bAP.ca_prox_apic.cai.maximum_voltage_from_voltagebase,mm.bAP.ca_prox_basal.cai.maximum_voltage_from_voltagebase,mm.bAP.ca_soma.cai.maximum_voltage_from_voltagebase,mm.bAP.ca_ais.cai.maximum_voltage_from_voltagebase,mm.Step_150.soma.v.voltage_base,mm.Step_150.soma.v.voltage_after_stim,mm.Step_150.soma.v.AP_amplitude,...,mm.Rin.soma.v.voltage_base,mm.RMP.soma.v.voltage_base,mm.RMP.soma.v.Spikecount,mm.IV_-100.soma.v.voltage_deflection,mm.IV_-100.soma.v.voltage_deflection_begin,mm.SpikeRec_600.soma.v.decay_time_constant_after_stim,mm.SpikeRec_600.soma.v.voltage_after_stim,mm.SpikeRec_600.soma.v.Spikecount,mm.RinHoldCurrent.soma.v.bpo_holding_current,mm.Threshold.soma.v.bpo_threshold_current
0,0.0,250.0,250.0,250.0,0.765146,0.709068,0.541270,0.168126,2.007624,2.972768,...,0.044521,0.050860,0.0,4.644261,4.376836,1.109476,0.423084,0.436691,0.136181,9.452932
1,0.0,250.0,250.0,250.0,0.765146,0.709068,0.541270,0.168126,2.007624,2.972768,...,0.044521,0.050860,0.0,4.644261,4.376836,1.109476,0.423084,0.436691,0.136181,9.452932
2,0.0,250.0,250.0,250.0,0.765146,0.709068,0.541270,0.168126,2.007624,2.972768,...,0.044521,0.050860,0.0,4.644261,4.376836,1.109476,0.423084,0.436691,0.136181,9.452932
3,0.0,250.0,250.0,250.0,0.765146,0.709068,0.541270,0.168126,2.007624,2.972768,...,0.044521,0.050860,0.0,4.644261,4.376836,1.109476,0.423084,0.436691,0.136181,9.452932
4,0.0,250.0,250.0,250.0,3.717283,1.075003,0.236818,0.142808,2.435834,0.694627,...,0.073232,0.058469,0.0,0.488456,0.121334,0.454181,1.289277,0.436691,0.247369,1.319094
5,0.0,250.0,250.0,250.0,3.555864,0.784416,0.728531,0.149000,2.396688,0.950158,...,0.066210,0.224511,0.0,0.028669,0.606706,1.953877,0.505147,1.684199,1.102784,1.615550
6,0.0,250.0,250.0,250.0,4.917991,0.964019,2.628989,0.266136,2.223090,2.668029,...,0.066626,0.105663,0.0,4.323116,4.036309,1.591309,2.263493,0.436691,1.207392,0.619084
7,0.0,250.0,250.0,250.0,5.629246,1.042212,0.511367,0.301844,2.668709,1.063198,...,0.107120,0.119117,0.0,0.658325,0.016197,1.811749,0.176152,0.436691,0.414817,0.486036
8,0.0,250.0,250.0,250.0,4.338694,1.030374,0.207082,0.174166,2.666370,0.203664,...,0.037671,0.261953,0.0,0.694408,0.083446,9.865182,1.431376,0.436691,0.116756,1.819813
9,0.0,250.0,250.0,250.0,0.765146,0.709068,0.541270,0.168126,2.007624,2.972768,...,0.044521,0.050860,0.0,4.644261,4.376836,1.109476,0.423084,0.436691,0.136181,9.452932


## <a id='select'>Select me-combinations</a>

In the last phase, the scores of the simulated me-combinations are compared with input threshold values. The successful combinations are stored in database.

In [8]:
select_config = 'select.json'
select_combos.select_combos(conf_filename=select_config, n_processes=None)

Reading configuration files
Reading tables from sqlite
Reading scores and score values from ./output/scores.sqlite ...
Checking if all combos have run
Start creation of ext_neurondb
Parallelising selection processing of e-models
Processing e-model cADpyr_L5TPC
Getting megating thresholds for emodel cADpyr_L5TPC
Applying megating to emodel cADpyr_L5TPC
Calculating median scores for emodel cADpyr_L5TPC
Received processed info from e-model cADpyr_L5TPC
All emodels processed, generating output files


/home/anil/miniconda3/envs/sscx-emodel-examples/lib/python3.9/site-packages/bluepymm/select_combos/reporting.py:377: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ext_neurondb = ext_neurondb.append(emodel_ext_neurondb_rows)
/home/anil/miniconda3/envs/sscx-emodel-examples/lib/python3.9/site-packages/bluepymm/select_combos/reporting.py:379: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  megate_passed_all = megate_passed_all.append(
/home/anil/miniconda3/envs/sscx-emodel-examples/lib/python3.9/site-packages/bluepymm/select_combos/reporting.py:381: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  median_scores = median_scores.append(emodel_median_scores)
/home/anil/miniconda3/envs/sscx-emodel-examples/lib/python3.9/site-packages/bl

Wrote me-type median scores to /home/anil/git-repos/SSCxEModelExamples/model_management/mm_run_minimal/output_select/extra_data/metype_median_scores.csv


/home/anil/miniconda3/envs/sscx-emodel-examples/lib/python3.9/site-packages/bluepymm/select_combos/reporting.py:311: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all axes decorations.
  plt.tight_layout()


Wrote pdf to /home/anil/git-repos/SSCxEModelExamples/model_management/mm_run_minimal/output_select/megating.pdf
Wrote extneurondb.dat to /home/anil/git-repos/SSCxEModelExamples/model_management/mm_run_minimal/output_select/extneurondb.dat
Wrote mecombo_emodel tsv to /home/anil/git-repos/SSCxEModelExamples/model_management/mm_run_minimal/output_select/mecombo_emodel.tsv
Wrote mecombo_release json to /home/anil/git-repos/SSCxEModelExamples/model_management/mm_run_minimal/output_select/mecombo_release.json


The selected combinations are written to the mecombo_emodel.tsv file.

In [9]:
pd.read_csv(Path("output_select") / "mecombo_emodel.tsv", sep="\t")

,morph_name,layer,fullmtype,etype,emodel,combo_name,threshold_current,holding_current
0,C060110A2,5,L5_TPC:A,cADpyr,cADpyr_L5TPC,cADpyr_L5TPC_L5_TPC:A_5_C060110A2,0.433659,-0.132662
1,C060114A5,5,L5_TPC:A,cADpyr,cADpyr_L5TPC,cADpyr_L5TPC_L5_TPC:A_5_C060114A5,1.199866,-0.170864
2,rp110616_C1_idB,5,L5_TPC:A,cADpyr,cADpyr_L5TPC,cADpyr_L5TPC_L5_TPC:A_5_rp110616_C1_idB,0.355185,-0.115984
3,rp110630_P1_idC,5,L5_TPC:A,cADpyr,cADpyr_L5TPC,cADpyr_L5TPC_L5_TPC:A_5_rp110630_P1_idC,0.480826,-0.168929
